# Seaborn Setup

In [1]:
height = 2.0 * 4
width = 3.5 * 4

fig_width_pt = 243.98238  # Get this from LaTeX using \showthe\columnwidth
inches_per_pt = 1.0/72.27               # Convert pt to inches
golden_mean = (math.sqrt(5)-1.0)/2.0         # Aesthetic ratio
fig_width = fig_width_pt*inches_per_pt  # width in inches
fig_height = fig_width*golden_mean       # height in inches
fig_size = [fig_width,fig_height]

rc = {  
#         'font.size': 32, # get this with \fontname\font\ at \the\fontdimen6\font
        'figure.titlesize': 10,
        'figure.titleweight': 700,
        'xtick.labelsize': 10,
        'ytick.labelsize': 10,
        'axes.labelsize': 10,
        'legend.fontsize': 10,
        'lines.linewidth': 1,
        'figure.figsize':fig_size
        }

sns.set_context('paper', rc=rc)


NameError: name 'math' is not defined

# Learning Effect

In [ ]:
%%R -i training_block_grouped

training_block_grouped$Participant = factor(training_block_grouped$Participant)

training_block_grouped$Block = factor(training_block_grouped$Block)
training_block_grouped$Condition = factor(training_block_grouped$Condition)

training_block_grouped$logTime = log(training_block_grouped$Total.Task.Time)

m <- ezANOVA(data = training_block_grouped, 
          wid = Participant, 
          dv = logTime, 
#           between = Condition,
          within = Block, 
          return_aov=TRUE)


pos = match(m$`Sphericity Corrections`$Effect, m$ANOVA$Effect) # positions of within-Ss efx in m$ANOVA
m$Sphericity$GGe.DFn = m$Sphericity$GGe * m$ANOVA$DFn[pos] # Greenhouse-Geisser
m$Sphericity$GGe.DFd = m$Sphericity$GGe * m$ANOVA$DFd[pos]
m$Sphericity$HFe.DFn = m$Sphericity$HFe * m$ANOVA$DFn[pos] # Huynh-Feldt
m$Sphericity$HFe.DFd = m$Sphericity$HFe * m$ANOVA$DFd[pos]

print(m)


# m <- aov(Total.Task.Time ~ Block + Error(Participant/Block), data=training_block_grouped)
# print(summary(m))
# print(TukeyHSD(m))

d = training_block_grouped
d$t = d$logTime

p12 = t.test(d[d$Block == 1,]$t, d[d$Block == 2,]$t,paired=TRUE)$p.value
p23 = t.test(d[d$Block == 2,]$t, d[d$Block == 3,]$t,paired=TRUE)$p.value
p34 = t.test(d[d$Block == 3,]$t, d[d$Block == 4,]$t,paired=TRUE)$p.value
p48 = t.test(d[d$Block == 4,]$t, d[d$Block == 8,]$t,paired=TRUE)$p.value
p89 = t.test(d[d$Block == 8,]$t, d[d$Block == 9,]$t,paired=TRUE)$p.value
p910 = t.test(d[d$Block == 9,]$t, d[d$Block == 10,]$t,paired=TRUE)$p.value
p1011 = t.test(d[d$Block == 10,]$t, d[d$Block == 11,]$t,paired=TRUE)$p.value

print(p.adjust(c(p12, p23,p34,p48,p89,p910,p1011), method='bonferroni'))

mm = unlist(m)

print(anova_to_latex(mm$Sphericity.GGe.DFn, mm$Sphericity.GGe.DFd, mm$`Sphericity.p[GG]`, mm$ANOVA.F, mm$ANOVA.ges))

# Expert Use

In [ ]:
training.groupby(['Condition'])[['Expert Use']].agg(['mean', 'std'])

In [ ]:
g = sns.barplot(x='Grouped', 
               hue='Condition', 
               y='Expert Use', 
               data=training_last_blocks_grouped, 
               )
g.get_figure().set_size_inches(6,2.5)
g.legend(loc='lower center',bbox_to_anchor=(0.5, -0.3), ncol=4)
g.set_ylabel("Expert Use")
g.set_xlabel("")
# g.legend(loc='lower right')
# g.set(ylim=(0, 0.7))
g.axes.set_label('')

g.set_yticklabels(['{:g}%'.format(x * 100) for x in g.get_yticks()])


g.get_figure().savefig('figures/expert_last_blocks_grouped.pdf', bbox_inches='tight')

In [ ]:
%%R -i training_last_blocks_grouped

training_last_blocks_grouped$Grouped = factor(training_last_blocks_grouped$Grouped)
training_last_blocks_grouped$Participant = factor(training_last_blocks_grouped$Participant)
training_last_blocks_grouped$Condition = factor(training_last_blocks_grouped$Condition)

grouped <- training_last_blocks_grouped[training_last_blocks_grouped$Grouped == 'Grouped',]
ungrouped <- training_last_blocks_grouped[training_last_blocks_grouped$Grouped == 'Ungrouped',]


grouped_m <- aov(Expert.Use ~ Condition, data=grouped)
ungrouped_m <- aov(Expert.Use ~ Condition, data=ungrouped)

print('-------------------')
print('----- Grouped -----')
print('-------------------')

print(summary(grouped_m))
print("Eta Squared")
print(eta_sq(grouped_m)[2]$etasq)
print(TukeyHSD(grouped_m))

print('---------------------')
print('----- Ungrouped -----')
print('---------------------')

print(summary(ungrouped_m))
print("Eta Squared")
print(eta_sq(ungrouped_m)[2]$etasq)
print(TukeyHSD(ungrouped_m))


qqnorm(residuals(grouped_m)); qqline(residuals(grouped_m))
qqnorm(residuals(ungrouped_m)); qqline(residuals(ungrouped_m))

# Time

In [ ]:
training.groupby(['Condition'])[['Total Task Time']].agg(['mean', 'std'])

In [ ]:
g = sns.barplot(x='Grouped', 
               hue='Condition', 
               y='Total Task Time', 
               data=training_last_blocks_grouped, 
               )

g.get_figure().set_size_inches(6,2.5)
g.legend(loc='lower center',bbox_to_anchor=(0.5, -0.3), ncol=4)
g.set_ylabel("Command Task Time (ms)")

# g.legend(loc='lower right')
g.axes.set_label('')
g.axes.set_xlabel('')
# g.set(ylim=(0, 5000))

g.get_figure().savefig('figures/time_last_blocks_grouped.pdf', bbox_inches='tight')

In [ ]:
%%R -i training_last_blocks_grouped

training_last_blocks_grouped$Grouped = factor(training_last_blocks_grouped$Grouped)
training_last_blocks_grouped$Participant = factor(training_last_blocks_grouped$Participant)
training_last_blocks_grouped$Condition = factor(training_last_blocks_grouped$Condition)

training_last_blocks_grouped$log_time = log(training_last_blocks_grouped$Total.Task.Time)

grouped <- training_last_blocks_grouped[training_last_blocks_grouped$Grouped == 'Grouped',]
ungrouped <- training_last_blocks_grouped[training_last_blocks_grouped$Grouped == 'Ungrouped',]

grouped_m <- aov(log_time ~ Condition, data=grouped)
ungrouped_m <- aov(log_time ~ Condition, data=ungrouped)

print('-------------------')
print('----- Grouped -----')
print('-------------------')

print(summary(grouped_m))
print(TukeyHSD(grouped_m))
print("Eta Squared")
print(eta_sq(grouped_m)[2]$etasq)

print(aov_to_latex(grouped_m))

print('---------------------')
print('----- Ungrouped -----')
print('---------------------')

print(summary(ungrouped_m))
print(TukeyHSD(ungrouped_m))

print("Eta Squared")
print(eta_sq(ungrouped_m)[2]$etasq)

print(aov_to_latex(ungrouped_m))


qqnorm(residuals(grouped_m)); qqline(residuals(grouped_m))
qqnorm(residuals(ungrouped_m)); qqline(residuals(ungrouped_m))

# for(condition in unique(grouped$Condition)) {
#     fit = fitdistr(grouped[grouped$Condition == condition,]$Total.Task.Time, "lognormal")$estimate
#     print(ks.test(grouped[grouped$Condition == condition,]$Total.Task.Time, "plnorm", meanlog=fit[1], sdlog=fit[2], exact=TRUE))
# }

# for(condition in unique(ungrouped$Condition)) {
#     fit = fitdistr(grouped[grouped$Condition == condition,]$Total.Task.Time, "lognormal")$estimate
#     print(ks.test(grouped[grouped$Condition == condition,]$Total.Task.Time, "plnorm", meanlog=fit[1], sdlog=fit[2], exact=TRUE))
# }


# Recall

In [ ]:
test.groupby(['Block'])[['Recall']].agg(['mean', 'std'])

In [ ]:
test.groupby(['Condition', 'Block'])[['Recall']].agg(['mean', 'std'])

In [ ]:

fig, (ax1, ax2) = plt.subplots(1, 2, sharey=True, figsize=[plt.rcParams["figure.figsize"][0], plt.rcParams["figure.figsize"][1] / 2])
plt.subplots_adjust(wspace=0.05, hspace=0.05)

g1 = sns.barplot(x='Grouped', 
               hue='Condition', 
               y='Recall', 
               data=test_grouped[test_grouped['Block'] == '10 Minutes'], 
                ax=ax1
               )
g1.set(ylim=(0, 8))
g1.axes.set_label('')
g1.set_xlabel("")
g1.set_title('10 Minutes')
g1.legend_.remove()

g2 = sns.barplot(x='Grouped', 
               hue='Condition', 
               y='Recall', 
               data=test_grouped[test_grouped['Block'] == '24 Hours'], 
               ax = ax2
               )

g2.set_xlabel("")
g2.set_ylabel("")
g1.set_title('Block 7 \n(after distraction task)')

g2.set_title('Block 13 \n(after 24 hours)')
g2.axes.set_label('')
g2.legend_.remove()
g1.set_ylabel('# of Items Recalled')
handles, labels = ax1.get_legend_handles_labels()
fig.subplots_adjust(bottom=0.26)

fig.legend(handles, labels, loc='lower center', ncol=4, )
fig.savefig('figures/recall_grouped.pdf', bbox_inches='tight')

In [ ]:
%%R -i test_grouped

test_grouped$Grouped = factor(test_grouped$Grouped)
test_grouped$Participant = factor(test_grouped$Participant)
test_grouped$Condition = factor(test_grouped$Condition)

grouped10min <- test_grouped[test_grouped$Grouped == 'Grouped' & test_grouped$Block == '10 Minutes',]
ungrouped10min <- test_grouped[test_grouped$Grouped == 'Ungrouped' & test_grouped$Block == '10 Minutes',]

grouped24hr <- test_grouped[test_grouped$Grouped == 'Grouped' & test_grouped$Block == '24 Hours',]
ungrouped24hr <- test_grouped[test_grouped$Grouped == 'Ungrouped' & test_grouped$Block == '24 Hours',]

grouped_m <- aov(Recall ~ Condition, data=grouped10min)
ungrouped_m <- aov(Recall ~ Condition, data=ungrouped10min)

print('-------------------')
print('----- Grouped 10 Minutes -----')
print('-------------------')

print(summary(grouped_m))
print("Eta Squared")
print(eta_sq(grouped_m)[2]$etasq)
print(TukeyHSD(grouped_m))
print(aov_to_latex(grouped_m))

print('---------------------')
print('----- Ungrouped 10 Minutes -----')
print('---------------------')

print(summary(ungrouped_m))
print("Eta Squared")
print(eta_sq(ungrouped_m)[2]$etasq)
print(TukeyHSD(ungrouped_m))
print(aov_to_latex(ungrouped_m))


qqnorm(residuals(grouped_m)); qqline(residuals(grouped_m))
qqnorm(residuals(ungrouped_m)); qqline(residuals(ungrouped_m))


grouped_m <- aov(Recall ~ Condition, data=grouped24hr)
ungrouped_m <- aov(Recall ~ Condition, data=ungrouped24hr)

print('-------------------')
print('----- Grouped 24 Hours -----')
print('-------------------')

print(summary(grouped_m))
print("Eta Squared")
print(eta_sq(grouped_m)[2]$etasq)
print(TukeyHSD(grouped_m))
print(aov_to_latex(grouped_m))


print('---------------------')
print('----- Ungrouped 24 Hours -----')
print('---------------------')

print(summary(ungrouped_m))
print("Eta Squared")
print(eta_sq(ungrouped_m)[2]$etasq)
print(TukeyHSD(ungrouped_m))
print(aov_to_latex(ungrouped_m))


qqnorm(residuals(grouped_m)); qqline(residuals(grouped_m))
qqnorm(residuals(ungrouped_m)); qqline(residuals(ungrouped_m))

# Errors

In [ ]:
training.groupby(['Condition'])[['Error Rate']].agg(['mean', 'std'])

In [ ]:
training[['Error Rate']].agg(['mean', 'std'])

In [ ]:
g = sns.barplot(x='Grouped', 
               hue='Condition', 
               y='Error Rate', 
               data=training_last_blocks_grouped, 
               )

g.get_figure().set_size_inches(6,2.5)
g.legend(loc='lower center',bbox_to_anchor=(0.5, -0.3), ncol=4)

g.set_xlabel("")
# g.legend(loc='upper right')
# g.set(ylim=(0, 0.35))
g.axes.set_label('')

g.set_yticklabels(['{:g}%'.format(x * 100) for x in g.get_yticks()])


g.get_figure().savefig('figures/error_last_blocks_grouped.pdf', bbox_inches='tight')

In [ ]:
%%R -i training_last_blocks_grouped

training_last_blocks_grouped$Grouped = factor(training_last_blocks_grouped$Grouped)
training_last_blocks_grouped$Participant = factor(training_last_blocks_grouped$Participant)
training_last_blocks_grouped$Condition = factor(training_last_blocks_grouped$Condition)

grouped <- training_last_blocks_grouped[training_last_blocks_grouped$Grouped == 'Grouped',]
ungrouped <- training_last_blocks_grouped[training_last_blocks_grouped$Grouped == 'Ungrouped',]


grouped_m <- aov(Error.Rate ~ Condition, data=grouped)
ungrouped_m <- aov(Error.Rate ~ Condition, data=ungrouped)


print('-------------------')
print('----- Grouped -----')
print('-------------------')

print(summary(grouped_m))
print("Eta Squared")
print(eta_sq(grouped_m)[2]$etasq)
print(TukeyHSD(grouped_m))

print('---------------------')
print('----- Ungrouped -----')
print('---------------------')

print(summary(ungrouped_m))
print("Eta Squared")
print(eta_sq(ungrouped_m)[2]$etasq)
print(TukeyHSD(ungrouped_m))

qqnorm(residuals(grouped_m)); qqline(residuals(grouped_m))
qqnorm(residuals(ungrouped_m)); qqline(residuals(ungrouped_m))

# unlisted = unlist(summary(ungrouped_m))
# print(unlisted)
# print(unlisted["Pr(>F)1"])

# Time Over Blocks

In [ ]:
g = sns.pointplot(hue="Condition", 
                   y="Total Task Time", 
                   x='Block',
#                    linestyles=['-', '--', '-.', ':'],
#                    markers=['+', 'o', 'D', 'x'],
                   data=training_block_grouped,
#                   dodge=0.2

                 )

g.get_figure().set_size_inches(6,2.5)
l = g.legend(loc='lower center',bbox_to_anchor=(0.5, -0.39), ncol=4)
g.set(ylabel='Command Task Time (ms)')
# g.legend(loc='upper right')
# g.set(ylim=(0, 0.35))
g.axes.set_label('')
l = g.get_lines()[0]

# g.get_lines()[7].remove()
# g.get_lines()[11].remove()
# g.get_lines()[15].remove()

g.get_figure().savefig('figures/time_over_blocks.pdf', bbox_inches='tight')

In [ ]:
dff = training.groupby(['Condition', 'Block'], as_index=False).agg(training_aggregations)


fig, ax = plt.subplots()
fig.set_size_inches(6,2.5)

for condition in training_block_grouped.Condition.cat.categories:
    df = dff[dff['Condition'] == condition]
    df1 = df[df['Block'] <= 4]
    df2 = df[df['Block'] >= 5]
    plt.plot(df1.Block, df1['Total Task Time'], marker='o')
    plt.plot(df2.Block - 3, df2['Total Task Time'], marker='o')
    

ax.set_xticks([1,2,3,4,5,6,7,8])
ax.set_xticklabels([1,2,3,4,8,9,10,11])
ax.set_xlabel('Block')

In [ ]:
training[training['Condition'] != 'Baseline'][['Cancellation Rate']].agg(['mean', 'std'])


In [ ]:
# fig, axes = plt.subplots(1,2)

sns.pointplot(x='Block',
              y='Cancellation Rate', 
              hue='Condition',
              data=training_block_grouped)
# sns.factorplot(x='Block',
#               y='Error Rate', 
#               hue='Condition',
#               data=training_block_grouped,
#               ax=axes[1])

# Physical Articulation

In [ ]:
training['modifier'] = training['shortcut'].str[1].fillna(0)

df = training.groupby(['Condition', 'Participant', 'Block', 'modifier'], as_index=False).agg(training_aggregations)

test['modifier'] = test['shortcut'].str[1].fillna(0)

df = test.groupby(['Condition', 'Participant', 'Block', 'modifier'], as_index=False).agg(test_aggregations)

g = sns.barplot(x='modifier', 
               hue='Condition', 
               y='Total Task Time', 
               data=training, 
               )


# test

In [ ]:
g = sns.barplot(x='modifier', 
               hue='Condition', 
               y='Recall', 
               data=test, 
               )

# Modality Over Blocks split Grouped

In [ ]:
# g = sns.factorplot(x='Block', 
#                hue='Selection Type', 
#                y='Number of Selections', 
# #                col='Grouped', 
#                col='Condition',  
#                data=modality, 
#                kind='bar', 
#                dodge=True,
#                size=height,
#                aspect=width/height)

# rcc = rc.copy()
# rc['linewidth']
# sns.set_context('paper', rc=rcc)


fig, axes = plt.subplots(1, 4, sharey=True, figsize=[12,2.2])
plt.subplots_adjust(wspace=0.05, hspace=0.05)

for ax, condition in zip(axes.reshape(-1),modality.Condition.cat.categories):
    g = sns.barplot(x='Block', 
                       hue='Selection Type', 
                       y='Number of Selections',  
                       data=modality_grouped[modality_grouped['Condition'] == condition],
#                         errwidth=1,
                       ax=ax)
    g.legend_.remove()
    g.axes.set_label('')
    g.set_ylabel("")
    g.set_title(condition)


# g.set_titles('{col_name} {row_name} Selections')
handles, labels = ax.get_legend_handles_labels()
fig.subplots_adjust(bottom=0.31)

fig.legend(handles, labels, loc='lower center', ncol=4)
fig.savefig('figures/modality_condition.pdf', bbox_inches='tight')

In [ ]:
fig, axes = plt.subplots(1, 4, sharey=True, figsize=[15,3])
plt.subplots_adjust(wspace=0.05, hspace=0.05)

for ax, condition in zip(axes.reshape(-1), modality.Condition.cat.categories):
    dff = modality[modality['Condition'] == condition]
    df = dff.groupby(['Block',  'Selection Type'], as_index=False).mean()
    dfsem = dff.groupby(['Block', 'Selection Type']).sem().reset_index()

    
    dff = modality[modality['Condition'] == condition].groupby(['Participant','Block', 'Selection Type'], as_index=False).mean()
    df = dff.groupby(['Block',  'Selection Type'], as_index=False).mean()
    dfsem = dff.groupby(['Block', 'Selection Type']).sem().reset_index()

    
#     display(dfsem)
#     display(df)
    
    blocks = [1,2,3,4,5,6,7,8]

    linear_menu = df[df['Selection Type'] == 'Linear Menu']['Number of Selections'].values
    shortcut_guidance = df[df['Selection Type'] == 'Shortcut Guidance']['Number of Selections'].values
    expert = df[df['Selection Type'] == 'Shortcut']['Number of Selections'].values
    
    linear_menu_sem = dfsem[dfsem['Selection Type'] == 'Linear Menu']['Number of Selections'].values
    shortcut_guidance_sem = dfsem[dfsem['Selection Type'] == 'Shortcut Guidance']['Number of Selections'].values
    expert_sem = dfsem[dfsem['Selection Type'] == 'Shortcut']['Number of Selections'].values
    
    linear = ax.bar(blocks, linear_menu, yerr=linear_menu_sem)
    shortcut = ax.bar(blocks, shortcut_guidance, bottom=linear_menu, yerr=shortcut_guidance_sem)
    expert = ax.bar(blocks, expert, bottom=linear_menu + shortcut_guidance, yerr=expert_sem)
    
    ax.set_label('')
    ax.set_title(condition)
    ax.set_xticks([1,2,3,4,5,6,7,8])
    ax.set_xticklabels([1,2,3,4,8,9,10,11])
    ax.set_xlabel('Block')

    ax.grid(False)
    ax.grid(axis='y')

fig.subplots_adjust(bottom=0.24)
axes[0].set_ylabel('Rate of Selections')
fig.legend([linear, shortcut, expert], ['Linear Menu', 'Shortcut Guidance', 'Shortcut'], loc='lower center', ncol=4)
fig.savefig('figures/modality_condition.pdf', bbox_inches='tight')

In [ ]:
# fig, axes = plt.subplots(1, 4, sharey=True, figsize=[15,3])
# plt.subplots_adjust(wspace=0.05, hspace=0.05)

# for ax, condition in zip(axes.reshape(-1), modality.Condition.cat.categories):    
#     dff = modality[modality['Condition'] == condition].groupby(['Participant', 'Selection Type'])
#     df = dff.mean().reset_index()
#     dfsem = dff.sem().reset_index()

    
# #     display(dfsem)
# #     display(df)
    
#     blocks = [1,2,3,4,5,6,7,8]

#     linear_menu = df[df['Selection Type'] == 'Linear Menu']['Number of Selections'].values
#     shortcut_guidance = df[df['Selection Type'] == 'Shortcut Guidance']['Number of Selections'].values
#     expert = df[df['Selection Type'] == 'Shortcut']['Number of Selections'].values
    
#     linear_menu_sem = dfsem[dfsem['Selection Type'] == 'Linear Menu']['Number of Selections'].values
#     shortcut_guidance_sem = dfsem[dfsem['Selection Type'] == 'Shortcut Guidance']['Number of Selections'].values
#     expert_sem = dfsem[dfsem['Selection Type'] == 'Shortcut']['Number of Selections'].values
    
#     linear = ax.bar(blocks, linear_menu, yerr=linear_menu_sem)
#     shortcut = ax.bar(blocks, shortcut_guidance, bottom=linear_menu, yerr=shortcut_guidance_sem)
#     expert = ax.bar(blocks, expert, bottom=linear_menu + shortcut_guidance, yerr=expert_sem)
    
#     ax.set_label('')
#     ax.set_title(condition)
#     ax.set_xticks([1,2,3,4,5,6,7,8])
# #     ax.set_xticklabels([1,2,3,4,8,9,10,11])
#     ax.set_xlabel('Participant')

#     ax.grid(False)
#     ax.grid(axis='y')

# fig.subplots_adjust(bottom=0.24)
# axes[0].set_ylabel('Percentage of Selections')
# fig.legend([linear, shortcut, expert], ['Linear Menu', 'Shortcut Guidance', 'Shortcut'], loc='lower center', ncol=4)
# # fig.savefig('figures/modality_condition.pdf', bbox_inches='tight')

# Subjective

## All

In [ ]:
g = sns.factorplot(hue="Condition", 
                   y="Rating", 
                   x='Measure',
                   kind='bar', 
                   data=subjective_melted, 
                   legend_out=False,
                   size=height, 
                   aspect=width/height)
g.set(ylim=[0,5])
g.set_axis_labels("", None)

g.set_xticklabels(rotation=45)

In [ ]:
df = subjective.copy()
del df['Participant']

df = df.groupby(['Condition'], as_index=False).agg(median_mad)
df = df.set_index('Condition')
display(df)

print(df.to_latex(bold_rows=True, column_format='l|c|c|c|c|c|c|c'))

## Ease Of Use

In [ ]:
g = sns.factorplot(x="Condition", 
                   y="Ease of Use", 
                   kind='bar', 
                   data=subjective, 
                   legend_out=False,
                   size=height, 
                   aspect=width/height)

g.set_axis_labels("", None)
g.set(ylim=[0,5])

In [ ]:
%%R -i subjective 

subjective$Condition = factor(subjective$Condition)

m <- art(Ease.of.Use ~ Condition, data=subjective)

df = data.frame(y=m$aligned.ranks$Condition, m$data)
m <- aov(data=df, y ~ Condition)
print(summary(m))
print(TukeyHSD(m))
print(aov_to_latex(m))

## Ease Of Learning

In [ ]:
g = sns.factorplot(x="Condition", 
                   y="Ease of Learning the Technique", 
                   kind='bar', 
                   data=subjective, 
                   legend_out=False)

g.set_axis_labels("", None)
g.set(ylim=[0,5])


In [ ]:
%%R -i subjective 

subjective$Condition = factor(subjective$Condition)

m <- art(Ease.of.Learning.the.Technique ~ Condition, data=subjective)
df = data.frame(y=m$aligned.ranks$Condition, m$data)
m <- aov(data=df, y ~ Condition)
print(summary(m))
print(TukeyHSD(m))
print(aov_to_latex(m))

## Posture

In [ ]:
# g = sns.factorplot(x="Posture", 
#                    y="Rating", 
#                    kind='bar',
#                    hue='Condition',
#                    data=posture, 
#                    legend_out=False,
#                    size=height, 
#                    aspect=width/height)

# g.set_axis_labels("", None)
# g.set(ylim=[0,5])
# g.savefig('subjective_learning.pdf')

df = posture[['Condition', 'Primary', 'Secondary','Tertiary','Quaternary']].groupby(['Condition'], as_index=False).agg(median_mad)
df = df.set_index('Condition')
display(df)

print(df.to_latex(bold_rows=True, column_format='l|c|c|c|c'))


In [ ]:
%%R -i posture

posture$Condition = factor(posture$Condition)

m <- art(Rating ~ Condition, data=posture)
df = data.frame(y=m$aligned.ranks$Condition, m$data)
m <- aov(data=df, y ~ Condition)
print(summary(m))
print(TukeyHSD(m))